<a href="https://colab.research.google.com/github/689photo/Colab_DeepLearning/blob/main/doat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content/'

!kaggle competitions download -c dogs-vs-cats-redux-kernels-edition

 99% 809M/814M [00:05<00:00, 136MB/s]
100% 814M/814M [00:05<00:00, 155MB/s]


In [4]:
!unzip -q train.zip -d .

In [5]:
import tensorflow as tf
import shutil

print(len(os.listdir('/content/train/')))
#폴더 안에 파일 갯수 세기

os.mkdir('/content/dataset/')
os.mkdir('/content/dataset/cat/')
os.mkdir('/content/dataset/dog/')
#폴더 만들기

for i in os.listdir('/content/train/'):
  if 'cat' in i:
    shutil.copyfile('/content/train/' + i, '/content/dataset/cat/' + i)
  if 'dog' in i:
    shutil.copyfile('/content/train/' + i, '/content/dataset/dog/' + i)
#폴더 안에 있는 고양이 / 강아지 사진 구분해서 폴더로 이동시키기

#tf.keras.preprocessing.image_dataset_from_directory()


25000


In [7]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/dataset/',
    image_size=(64,64),
    batch_size=64,
    subset='training',
    validation_split=0.2,
    seed=1234
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/dataset/',
    image_size=(64,64),
    batch_size=64,
    subset='validation',
    validation_split=0.2,
    seed=1234
)

print(train_ds)

def 전처리함수(i, 정답):
  i = tf.cast(i/255.0, tf.float32)
  return i, 정답

train_ds = train_ds.map(전처리함수)
val_ds = val_ds.map(전처리함수)

#import matplotlib.pyplot as plt
#
#for i, 정답 in train_ds.take(1):
#  print(i)
#  print(정답)
#  plt.imshow(i[10].numpy().astype('uint8'))
#  plt.show()

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
<BatchDataset element_spec=(TensorSpec(shape=(None, 64, 64, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


In [8]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32,(3,3), padding="same", activation='relu', input_shape=(64,64,3)),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(64,(3,3), padding="same", activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Dropout(0.2),
    #overfitting을 완화하기 위해서 Dropout
    tf.keras.layers.Conv2D(128,(3,3), padding="same", activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation="sigmoid"),
])

model.summary()

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])
model.fit(train_ds, validation_data=val_ds, epochs=5)


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 64, 64, 32)        896       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 32, 32, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 32, 32, 64)        18496     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 16, 16, 64)       0         
 2D)                                                             
                                                                 
 dropout_2 (Dropout)         (None, 16, 16, 64)        0         
                                                                 
 conv2d_5 (Conv2D)           (None, 16, 16, 128)      